## 用决策树来分类贷款是否优良

[LendingClub](https://www.lendingclub.com/) 是一家贷款公司. 在本次作业中,我们需要手动实现决策树来预测一份贷款是否安全，并对比不同复杂度下决策树的表现

In [3]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree
from sklearn import metrics




## 读取数据

In [4]:
data = pd.read_csv(os.path.join("../input", "loan_sub.csv"), sep=',')

/Users/yongqiangshen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 打印可用特征

In [5]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

## 预处理预测目标

预测目标是一列'bad_loans'的数据。其中**1**表示的是不良贷款，**0**表示的是优质贷款。

将预测目标处理成更符合直觉的标签，创建一列 `safe_loans`. 并且: 

* **+1** 表示优质贷款, 
* **-1** 表示不良贷款. 

In [6]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
data['safe_loans'] = data['bad_loans'].apply(lambda x : +1 if x==0 else -1)
data = data.drop('bad_loans', axis=1)

## 打印优质贷款与不良贷款的比例

In [7]:
data['safe_loans'].value_counts(normalize=True)

 1    0.811185
-1    0.188815
Name: safe_loans, dtype: float64

#### 这是一个不平衡数据， 好的贷款远比坏的贷款要多. 

## 选取用于预测的特征

In [8]:
cols = ['grade', 'term','home_ownership', 'emp_length']
target = 'safe_loans'

data = data[cols + [target]]
data.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


## 创建更为平衡的数据集  

* 对占多数的标签进行下采样  
* 注意有很多方法处理不平衡数据，下采样只是其中之一

In [9]:
data['safe_loans'].value_counts()


 1    99457
-1    23150
Name: safe_loans, dtype: int64

In [10]:

# use the percentage of bad and good loans to undersample the safe loans.
bad_ones = data[data[target] == -1]
safe_ones = data[data[target] == 1]
percentage = len(bad_ones)/float(len(safe_ones))

risky_loans = bad_ones
safe_loans = safe_ones.sample(frac=percentage, random_state=33)

# combine two kinds of loans
data_set = pd.concat([risky_loans, safe_loans], axis=0)

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [11]:
data_set[target].value_counts(normalize=True)

-1    0.5
 1    0.5
Name: safe_loans, dtype: float64

## Preprocessing your features

In [12]:
def dummies(data, columns=['pclass','name_title','embarked', 'sex']):
    for col in columns:
        data[col] = data[col].apply(lambda x: str(x))
        new_cols = [col + '_' + i for i in data[col].unique()]
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col)[new_cols]], axis=1)
        del data[col]
    return data

In [13]:
#grade, home_ownership, target
cols = ['grade', 'term','home_ownership', 'emp_length']
data_set = dummies(data_set, columns=cols)
data_set.head()

,safe_loans,grade_C,grade_F,grade_B,grade_D,grade_A,grade_E,grade_G,term_ 60 months,term_ 36 months,...,emp_length_3 years,emp_length_10+ years,emp_length_1 year,emp_length_9 years,emp_length_2 years,emp_length_8 years,emp_length_7 years,emp_length_5 years,emp_length_nan,emp_length_6 years
1,-1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10,-1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12,-1,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


## 将数据分成训练集和测试集

重要的事情说三遍!!  

**把你的爪子从TEST DATA上拿开!!**   
**把你的爪子从TEST DATA上拿开!!**  
**把你的爪子从TEST DATA上拿开!!**  


In [14]:
train_data, test_data = train_test_split(data_set, test_size=0.2, random_state=33)

## 建自己的决策树!

- 第一步，实现统计错误和计算熵的两个函数count_errors(), entropy()

In [15]:
def count_errors(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    
    positive_ones = labels_in_node.apply(lambda x: x==1).sum()
    negative_ones = labels_in_node.apply(lambda x: x==-1).sum()
    
    return min(positive_ones, negative_ones)


def entropy(y):
    # 二分类问题: 0 or 1
    n = len(y)
    s1 = (y==1).sum()
    if s1 == 0 or s1 == n:
        return 0
    
    p1 = float(s1) / n
    p0 = 1 - p1
    return -p0 * np.log2(p0) - p1 * np.log2(p1)

* 第二步, 分别根据错误率和信息增益实现选择最佳特征的函数  
1. best_split()： 根据犯错的个数选择最佳特征
2. best_split_entropy(): 根据信息增益选择最佳特征  
3. 约定左边对应0， 右边对应1

In [16]:
def best_split(data, features, target):
    # return the best feature
    best_feature = None
    best_error = 2.0 
    num_data_points = float(len(data))  

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = data[data[feature] == 0]
        
        # 右分支对应当前特征为1的数据点
        right_split = data[data[feature] == 1] 
        
        # 计算左边分支里犯了多少错
        left_misses = count_errors(left_split[target])            

        # 计算右边分支里犯了多少错
        right_misses = count_errors(right_split[target])
            
        # 计算当前划分之后的分类犯错率
        error = (left_misses + right_misses) * 1.0 / num_data_points

        # 更新应选特征和错误率，注意错误越低说明该特征越好
        if error < best_error:
            best_error = error
            best_feature = feature
    return best_feature


def best_split_entropy(data, features, target):
    
    best_feature = None
    best_info_gain = float('-inf') 
    num_data_points = float(len(data))
    # 计算划分之前数据集的整体熵值
    entropy_original = entropy(data[target])

    for feature in features:
        
        # 左分支对应当前特征为0的数据点
        left_split = data[data[feature] == 0]
        
        # 右分支对应当前特征为1的数据点
        right_split = data[data[feature] == 1] 
        
        # 计算左边分支的熵值
        left_entropy = entropy(left_split[target])            

        # 计算右边分支的熵值
        right_entropy = entropy(right_split[target])
            
        # 计算左边分支与右分支熵值的加权和（数据集划分后的熵值）
        entropy_split = len(left_split) / num_data_points * left_entropy + len(right_split) / num_data_points * right_entropy
        
        # 计算划分前与划分后的熵值差得到信息增益
        info_gain = entropy_original - entropy_split

        # 更新最佳特征和对应的信息增益的值
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature
    return best_feature
    

* 定义树上的节点，每个树节点应该包含如下信息:  

   1 is_leaf: True/False  表示当前节点是否为叶子节点  
   
   2 prediction: 当前节点做全民公投的预测结果
   
   3 left: 左子树  
   
   4 right: 右子树 
   
   5 split_feature: 当前节点用来划分数据时所采用的特征

In [17]:
class TreeNode:
    def __init__(self, is_leaf, prediction, split_feature):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.split_feature = split_feature
        self.left = None
        self.right = None
        

In [18]:
def create_leaf(target_values):
    # 用于创建叶子的函数
    
    # 初始化一个树节点
    leaf = TreeNode(True, None, None)
    
    # 统计当前数据集里标签为+1和-1的个数，较大的那个即为当前节点的预测结果
    num_positive_ones = len(target_values[target_values == +1])
    num_negative_ones = len(target_values[target_values == -1])
    
    if num_positive_ones > num_negative_ones:
        leaf.prediction = 1
    else:
        leaf.prediction = -1
        
    # 返回叶子        
    return leaf 

* 探索三种不同的终止划分数据集的条件  
  
  termination 1, 当错误率降到min_error以下, 终止划分并返回叶子节点  
  
  termination 2, 当特征都用完了, 终止划分并返回叶子节点  
  
  termination 3, 当树的深度等于最大max_depth时, 终止划分并返回叶子节点

In [19]:
def create_tree(data, features, target, current_depth = 0, max_depth = 10, min_error=0):
    # 拷贝以下可用特征
    remaining_features = features[:]
    
    target_values = data[target]
    
    # termination 1
    if count_errors(target_values) <= min_error:
        print("Termination 1 reached.")     
        return create_leaf(target_values)
    
    # termination 2
    if len(remaining_features) == 0:
        print("Termination 2 reached.")    
        return create_leaf(target_values)    
    
    # termination 3
    if current_depth >= max_depth: 
        print("Termination 3 reached.")
        return create_leaf(target_values)

    
    
    # 选出最佳当前划分特征
    #split_feature = best_split(data, features, target)   #根据正确率划分
    split_feature = best_split_entropy(data, features, target)  # 根据信息增益来划分
    
    # 选出最佳特征后，该特征为0的数据分到左边，该特征为1的数据分到右边
    left_split = data[data[split_feature] == 0]
    right_split = data[data[split_feature] == 1]
    
    # 剔除已经用过的特征
    remaining_features = remaining_features.drop(split_feature)
    print("Split on feature %s. (%s, %s)" % (split_feature, str(len(left_split)), str(len(right_split))))
    
    # 如果当前数据全部划分到了一边，直接创建叶子节点返回即可
    if len(left_split) == len(data):
        print("Perfect split!")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Perfect split!")
        return create_leaf(right_split[target])
        
    # 递归上面的步骤
    left_tree = create_tree(left_split, remaining_features, target, current_depth + 1, max_depth,min_error)        
    right_tree = create_tree(right_split,remaining_features,target, current_depth + 1, max_depth,min_error)
    
    
    #生成当前的树节点
    result_node = TreeNode(False, None, split_feature)
    result_node.left = left_tree
    result_node.right = right_tree
    return result_node

In [20]:
def predict(tree, x, annotate = False):   
    # 如果已经是叶子节点直接返回叶子节点的预测结果
    if tree.is_leaf:
        if annotate: 
            print("leaf node, predicting %s" % tree.prediction)
        return tree.prediction 
    else:
        # 查询当前节点用来划分数据集的特征
        split_feature_value = x[tree.split_feature]
        
        if annotate: 
            print("Split on %s = %s" % (tree.split_feature, split_feature_value))
        if split_feature_value == 0:
            #如果数据在该特征上的值为0，交给左子树来预测
            return predict(tree.left, x, annotate)
        else:
            #如果数据在该特征上的值为0，交给右子树来预测
            return predict(tree.right, x, annotate)

In [21]:
def evaluate_accuracy(tree, data):
    # 将predict函数应用在数据data的每一行
    prediction = data.apply(lambda row: predict(tree, row), axis=1)
    
    # 返回正确率
    accuracy = (prediction == data['safe_loans']).sum() * 1.0 / len(data)
    return accuracy

In [22]:
# 所有可用特征
features = train_data.columns.drop(target)

In [23]:
my_decesion_tree = create_tree(train_data, features, target, current_depth = 0, max_depth = 10)

Split on feature grade_A. (31776, 5264)
Split on feature grade_B. (21587, 10189)
Split on feature grade_C. (12308, 9279)
Split on feature grade_D. (5553, 6755)
Split on feature term_ 60 months. (1743, 3810)
Split on feature grade_E. (459, 1284)
Split on feature emp_length_10+ years. (358, 101)
Split on feature emp_length_6 years. (328, 30)
Split on feature home_ownership_OTHER. (325, 3)
Split on feature emp_length_4 years. (297, 28)
Termination 3 reached.
Termination 3 reached.
Termination 1 reached.
Split on feature home_ownership_MORTGAGE. (23, 7)
Split on feature home_ownership_OWN. (21, 2)
Termination 3 reached.
Termination 3 reached.
Termination 1 reached.
Split on feature grade_F. (25, 76)
Split on feature home_ownership_OWN. (21, 4)
Split on feature home_ownership_RENT. (13, 8)
Termination 3 reached.
Termination 3 reached.
Split on feature grade_G. (0, 4)
Perfect split!
Split on feature home_ownership_RENT. (47, 29)
Split on feature home_ownership_OWN. (39, 8)
Termination 3 reac

Split on feature term_ 60 months. (9134, 1055)
Split on feature emp_length_nan. (8739, 395)
Split on feature home_ownership_MORTGAGE. (4700, 4039)
Split on feature emp_length_2 years. (4091, 609)
Split on feature emp_length_6 years. (3795, 296)
Split on feature emp_length_4 years. (3405, 390)
Split on feature home_ownership_OTHER. (3392, 13)
Split on feature emp_length_8 years. (3207, 185)
Termination 3 reached.
Termination 3 reached.
Split on feature emp_length_10+ years. (7, 6)
Termination 3 reached.
Termination 1 reached.
Split on feature home_ownership_RENT. (42, 348)
Split on feature grade_C. (42, 0)
Perfect split!
Split on feature grade_C. (348, 0)
Perfect split!
Split on feature home_ownership_OTHER. (294, 2)
Split on feature home_ownership_RENT. (44, 250)
Split on feature grade_C. (44, 0)
Perfect split!
Split on feature grade_C. (250, 0)
Perfect split!
Termination 1 reached.
Split on feature home_ownership_OWN. (547, 62)
Split on feature home_ownership_RENT. (1, 546)
Terminatio

In [24]:
evaluate_accuracy(my_decesion_tree, test_data)

0.6193304535637149

### 决策树复杂度的探讨

In [25]:
def count_leaves(tree):
    if tree.is_leaf:
        return 1
    return count_leaves(tree.left) + count_leaves(tree.right)

#### 探索不同树深度对决策树的影响  

- max_depth = 3  
- max_depth = 7  
- max_depth = 15


In [26]:
model_1 = create_tree(train_data, features, target, current_depth = 0, max_depth = 3)
model_2 = create_tree(train_data, features, target, current_depth = 0, max_depth = 7)
model_3 = create_tree(train_data, features, target, current_depth = 0, max_depth = 15)

Split on feature grade_A. (31776, 5264)
Split on feature grade_B. (21587, 10189)
Split on feature grade_C. (12308, 9279)
Termination 3 reached.
Termination 3 reached.
Split on feature term_ 60 months. (9134, 1055)
Termination 3 reached.
Termination 3 reached.
Split on feature emp_length_nan. (5037, 227)
Split on feature home_ownership_RENT. (3142, 1895)
Termination 3 reached.
Termination 3 reached.
Split on feature term_ 60 months. (220, 7)
Termination 3 reached.
Termination 3 reached.
Split on feature grade_A. (31776, 5264)
Split on feature grade_B. (21587, 10189)
Split on feature grade_C. (12308, 9279)
Split on feature grade_D. (5553, 6755)
Split on feature term_ 60 months. (1743, 3810)
Split on feature grade_E. (459, 1284)
Split on feature emp_length_10+ years. (358, 101)
Termination 3 reached.
Termination 3 reached.
Split on feature emp_length_nan. (1223, 61)
Termination 3 reached.
Termination 3 reached.
Split on feature home_ownership_MORTGAGE. (1919, 1891)
Split on feature emp_le

Split on feature home_ownership_RENT. (1, 19)
Termination 1 reached.
Split on feature grade_G. (19, 0)
Perfect split!
Split on feature grade_F. (0, 2)
Perfect split!
Termination 1 reached.
Split on feature grade_F. (25, 76)
Split on feature home_ownership_OWN. (21, 4)
Split on feature home_ownership_RENT. (13, 8)
Split on feature grade_G. (0, 13)
Perfect split!
Split on feature grade_G. (0, 8)
Perfect split!
Split on feature grade_G. (0, 4)
Perfect split!
Split on feature home_ownership_RENT. (47, 29)
Split on feature home_ownership_OWN. (39, 8)
Split on feature grade_G. (39, 0)
Perfect split!
Split on feature grade_G. (8, 0)
Perfect split!
Split on feature grade_G. (29, 0)
Perfect split!
Split on feature emp_length_nan. (1223, 61)
Split on feature emp_length_1 year. (1117, 106)
Split on feature home_ownership_OTHER. (1113, 4)
Split on feature emp_length_2 years. (963, 150)
Split on feature home_ownership_MORTGAGE. (630, 333)
Split on feature emp_length_10+ years. (475, 155)
Split on f

Split on feature home_ownership_RENT. (4, 1474)
Split on feature grade_F. (4, 0)
Perfect split!
Split on feature emp_length_< 1 year. (1182, 292)
Termination 3 reached.
Termination 3 reached.
Split on feature emp_length_9 years. (248, 12)
Split on feature emp_length_10+ years. (130, 118)
Termination 3 reached.
Termination 3 reached.
Split on feature grade_F. (12, 0)
Perfect split!
Split on feature home_ownership_RENT. (25, 211)
Split on feature grade_F. (25, 0)
Perfect split!
Split on feature grade_F. (211, 0)
Perfect split!
Split on feature home_ownership_RENT. (23, 131)
Split on feature grade_F. (23, 0)
Perfect split!
Split on feature grade_F. (131, 0)
Perfect split!
Split on feature home_ownership_OTHER. (187, 1)
Split on feature home_ownership_RENT. (26, 161)
Split on feature grade_F. (26, 0)
Perfect split!
Split on feature grade_F. (161, 0)
Perfect split!
Termination 1 reached.
Split on feature home_ownership_RENT. (42, 263)
Split on feature home_ownership_OWN. (2, 40)
Termination

Split on feature grade_F. (16, 0)
Perfect split!
Split on feature grade_F. (54, 0)
Perfect split!
Split on feature home_ownership_RENT. (5, 44)
Split on feature grade_F. (5, 0)
Perfect split!
Split on feature grade_F. (44, 0)
Perfect split!
Split on feature home_ownership_RENT. (3, 33)
Termination 1 reached.
Split on feature grade_F. (33, 0)
Perfect split!
Split on feature home_ownership_RENT. (10, 22)
Split on feature grade_F. (10, 0)
Perfect split!
Split on feature grade_F. (22, 0)
Perfect split!
Split on feature home_ownership_RENT. (5, 82)
Split on feature grade_F. (5, 0)
Perfect split!
Split on feature grade_F. (82, 0)
Perfect split!
Split on feature home_ownership_RENT. (7, 56)
Split on feature grade_F. (7, 0)
Perfect split!
Split on feature grade_F. (56, 0)
Perfect split!
Split on feature emp_length_2 years. (1263, 73)
Split on feature emp_length_1 year. (1210, 53)
Split on feature emp_length_7 years. (1131, 79)
Split on feature emp_length_3 years. (1047, 84)
Split on feature em

Split on feature home_ownership_OWN. (39, 7)
Split on feature grade_C. (39, 0)
Perfect split!
Split on feature grade_C. (7, 0)
Perfect split!
Split on feature grade_C. (35, 0)
Perfect split!
Split on feature home_ownership_RENT. (35, 22)
Split on feature home_ownership_OWN. (30, 5)
Split on feature grade_C. (30, 0)
Perfect split!
Split on feature grade_C. (5, 0)
Perfect split!
Split on feature grade_C. (22, 0)
Perfect split!
Split on feature emp_length_nan. (5037, 227)
Split on feature home_ownership_RENT. (3142, 1895)
Split on feature term_ 60 months. (3056, 86)
Split on feature emp_length_2 years. (2781, 275)
Split on feature home_ownership_OWN. (2430, 351)
Split on feature emp_length_4 years. (2263, 167)
Split on feature emp_length_3 years. (2070, 193)
Split on feature emp_length_6 years. (1900, 170)
Split on feature emp_length_< 1 year. (1724, 176)
Split on feature home_ownership_MORTGAGE. (3, 1721)
Split on feature emp_length_10+ years. (1, 2)
Termination 1 reached.
Split on featu

In [27]:
print("model_1 training accuracy :", evaluate_accuracy(model_1, train_data))
print("model_2 training accuracy :", evaluate_accuracy(model_2, train_data))
print("model_3 training accuracy :", evaluate_accuracy(model_3, train_data))

model_1 training accuracy : 0.6173326133909287
model_2 training accuracy : 0.6229481641468683
model_3 training accuracy : 0.6266198704103672


In [28]:
print("model_1 testing accuracy :", evaluate_accuracy(model_1, test_data))
print("model_2 testing accuracy :", evaluate_accuracy(model_2, test_data))
print("model_3 testing accuracy :", evaluate_accuracy(model_3, test_data))

model_1 testing accuracy : 0.6173866090712743
model_2 testing accuracy : 0.6206263498920086
model_3 testing accuracy : 0.6187904967602592


In [29]:
print("model_1 complexity is: ", count_leaves(model_1))
print("model_2 complexity is: ", count_leaves(model_2))
print("model_3 complexity is: ", count_leaves(model_3))

model_1 complexity is:  8
model_2 complexity is:  74
model_3 complexity is:  384
